<a href="https://colab.research.google.com/github/pld000/z3/blob/main/task-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


По тропинке вдоль кустов
Шли 11 хвостов.
Сосчитать я также смог,
Что шагало 30 ног.
Это вместе шли куда-то
Петухи и поросята.
А теперь вопрос таков:
Сколько было петухов?
И узнать я был бы рад,
Сколько было поросят?

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 30.5 MB/s eta 0:00:00


In [2]:
from z3 import *

In [3]:
chickenCount, pigCount = Ints('chickenCount pigCount')
equations = [
    chickenCount * 2 + pigCount * 4 == 30,
    chickenCount + pigCount == 11,
    chickenCount >= 1,
    pigCount >= 1
]
solve(equations)

[pigCount = 4, chickenCount = 7]


Три купца разделили между собой 100 монет. Первый взял в 2 раза больше, чем второй, а третий на 10 меньше, чем первый. Сколько монет взял каждый?

In [1]:
a, b, c = Ints('a b c')
solve(
    a + b + c == 100,
    a == 2 * b,
    c == a - 10,
    a > 0, b > 0, c > 0
)

NameError: name 'Ints' is not defined